# ASPIICS L3 batch processing

In [ ]:
from pathlib import Path
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

def process_l3_image(files, output_dir):
    # print(f"Processing L3 image: {[Path(file).name for file in files]}")
    cmd = [
        "python",
        "l3_processor/l3_merge.py",
        str(files[0]),
        str(files[1]),
        str(files[2]),
        "--coalign",
        "--soft_merge",
        "--outdir",
        str(output_dir),
    ]
    # print(" ".join(cmd))
    result = subprocess.run(cmd, check=True, shell=True)
    return result

def group_l2_files(files):
    # Group files by common pattern in filename
    groups = {}
    for file in files:
        file_stem = Path(file).stem
        key = file_stem.split("_")[-2][:-2]  # remove last character to group
        if key not in groups:
            groups[key] = []
        groups[key].append(file)
    return groups

## Adjust your directories below:

In [ ]:
# ***************** Input images *******************
input_files = list(Path('PATH/TO/YOUR/L2/FILES/').rglob('aspiics_wb_l2*.fits'))

output_dir = Path('PATH/TO/YOUR/OUTPUT/L3/DIRECTORY/')

# Number of threads for parallel processing
threads = 8

In [ ]:
output_dir.mkdir(parents=True, exist_ok=True)
input_files.sort()

# group files by name pattern (assuming 3 files per group) eg. aspiics_wb_l2_1110800000021*

groups = group_l2_files(input_files)

print(f"Found {len(groups)} groups of L2 files to process.")


# Process images in parallel using threads:
with ThreadPoolExecutor(max_workers=threads) as executor:
    futures = {executor.submit(process_l3_image, group, Path("Y:/06_SpaceStuff/Spacecraft/Proba3/processed/Orbit355/L3_WB_prefilter")): group for group in groups.values() if len(group) == 3}
    
    with tqdm(total=len(futures), desc="Processing images") as pbar:
        for future in as_completed(futures):
            try:
                future.result()  # to catch exceptions if any
            except Exception as e:
                print(f"Error processing {futures[future]}: {e}")
            pbar.update(1)